In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import auc
from sklearn.metrics import matthews_corrcoef, make_scorer
from sklearn import metrics

### Load Data

In [2]:
#train_numeric = pd.read_csv('data/train_numeric.csv')
#train_numeric

In [3]:
train_numeric_reduced=pd.read_csv('data/shared_data/numeric/train_numeric_f65_new2.csv')

In [4]:
train_numeric_reduced = train_numeric_reduced.drop(train_numeric_reduced.columns[[0]], axis=1)
#train_numeric_reduced.head()

In [64]:
#train_numeric_reduced = train_numeric_reduced.apply(np.random.permutation)

In [65]:
train_numeric_reduced

,Id,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,Response
0,523236.0,-0.643160,1.543700,0.042901,-0.225260,-0.100790,-0.413890,-0.125390,-0.588210,1.021700,...,-5.308200,0.095186,2.966800,-0.036201,-1.295200,0.721390,2.269900,-2.002400,-0.082109,0
1,818981.0,-0.729360,-0.210770,0.220740,0.227370,0.074290,-0.345130,-0.354560,-0.431730,-1.589600,...,0.351940,-1.097000,-0.744940,0.085015,0.982910,0.823380,1.286400,1.651000,-0.104600,0
2,1510592.0,0.305440,-1.491700,-1.805600,-0.038606,1.035400,-0.076997,-0.025700,1.158700,-0.074695,...,-0.359350,-1.877300,3.245900,2.343200,-1.652100,-0.455530,1.660400,3.992400,0.343700,0
3,2352302.0,-1.183000,-1.176400,0.070867,-0.076601,-0.162860,-0.105140,0.107890,0.393450,-0.351840,...,-0.327210,2.145900,0.755610,1.992400,-2.143700,0.042068,0.578630,-1.696700,-7.407700,0
4,280067.0,-1.698900,-2.265400,-0.331600,-0.032946,0.445020,-10.563000,-0.314100,0.096619,0.732910,...,-0.267930,-0.102550,-0.404600,3.032200,0.848820,0.117450,-2.557600,-5.963700,0.041169,0
5,2292917.0,0.562430,0.699070,0.279380,-0.101940,-0.162660,0.521630,-0.196020,0.327780,0.061156,...,0.554430,-1.445900,1.199800,0.058493,-0.588140,0.160930,-1.108800,-3.956800,0.767380,0
6,1369671.0,-0.611600,-0.464990,0.222400,4.364700,-2.298700,0.003914,0.170120,-0.082980,0.503140,...,0.350140,2.367300,0.374760,0.553970,1.895000,0.099635,1.287500,3.408100,0.277550,0
7,929012.0,-0.915870,0.126150,-0.087524,-0.244710,0.228720,-1.488800,-0.261780,-0.940640,-0.155020,...,-0.270440,1.021000,-0.732630,-0.801010,2.740000,-0.477510,-1.817800,-3.987700,-0.085351,0
8,523327.0,-1.793400,-2.417500,0.100870,-0.139330,0.799560,-0.299540,-0.083231,0.082833,2.951000,...,0.443100,1.658200,2.610200,2.101300,1.270400,0.798500,-0.544400,-5.995600,0.127840,0
9,1844636.0,-2.453300,-0.282090,0.076496,0.111250,0.950960,-0.671090,0.272480,-0.423190,0.257220,...,0.290300,0.591540,-0.349940,-1.472200,-0.513760,0.422550,-0.228130,-4.620400,0.160040,0


In [158]:
train_numeric_reduced.to_csv('data/shared_data/numeric/train_numeric_f65_new3.csv', index=False)

In [76]:
pos_data = train_numeric_reduced.loc[train_numeric_reduced['Response'] == 1] 
neg_data = train_numeric_reduced.loc[train_numeric_reduced['Response'] == 0]
pos_data = pos_data.apply(np.random.permutation)
neg_data = neg_data.apply(np.random.permutation)
np.size(pos_data,0), np.size(neg_data,0)

(6879, 1176868)

In [77]:
neg_data

,Id,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,Response
0,1976865.0,-0.303610,-0.857920,0.166340,-0.104850,-1.374800,-0.911240,-0.109820,-0.473700,-0.32037,...,0.906320,2.093900,0.658430,3.516800,0.352790,0.630220,2.731400,-4.255900,-0.319320,0
1,161530.0,1.429100,1.674200,-1.428900,0.364750,0.274070,0.200420,-0.054901,0.191420,0.57016,...,-0.918430,1.608100,-1.079400,1.619500,-0.218740,0.288400,-1.549400,-0.065098,-0.442170,0
2,2359506.0,2.189300,-1.561400,-0.136380,-0.650750,0.364140,-0.910120,0.317140,-1.609200,0.34889,...,-0.025098,1.150300,-0.223650,-1.723000,2.137600,0.068844,1.390400,-4.052300,1.291000,0
3,603589.0,-0.317010,-1.298600,-0.399550,-0.349750,0.525960,1.696000,0.137510,0.331550,-1.52620,...,0.192110,-1.372800,-1.017000,-0.822530,-2.257000,1.196700,0.751540,6.654100,-0.659660,0
4,1352806.0,-2.801500,1.472100,0.455720,-0.063933,0.918790,0.300780,0.188030,-0.274380,-1.70980,...,-0.091288,0.867300,-0.207850,-3.834700,-0.219150,-0.096311,1.225100,-3.054300,0.796940,0
5,208535.0,1.643700,-0.076288,0.641370,-0.313600,-0.034460,0.580200,-0.329740,-0.443290,-0.63371,...,-0.556080,3.913400,-0.268860,-1.312200,0.268430,1.217700,1.533700,3.783500,1.225800,0
6,1807757.0,-1.647100,0.762360,-0.548780,-0.049677,0.420750,-0.441200,-0.201650,1.152300,-1.57050,...,-0.310460,1.673000,1.100600,3.121400,12.850000,0.219730,0.736240,3.375500,0.058360,0
7,16774.0,1.842200,-3.217000,-0.514680,0.039836,-0.803230,-0.316830,0.145240,-2.022000,0.90889,...,-0.081876,3.359900,-0.507830,1.545700,-2.689000,-0.403770,2.455900,-4.770300,0.100060,0
8,1738250.0,-0.724900,-1.117800,-0.065993,0.022245,0.995220,-0.258550,0.131010,0.845680,-1.52890,...,-12.439000,-1.729300,13.118000,-2.421200,0.534500,-0.357970,-1.332200,1.313300,0.042200,0
9,1190010.0,3.637200,-1.517100,0.024552,0.130390,-4.234700,0.399150,0.257830,-0.968500,-1.24900,...,0.243390,-0.352660,0.897940,1.311100,-0.137800,1.044800,-0.498340,-4.408100,0.170890,0


In [85]:
pos_num = np.size(pos_data,0)
neg_num = np.size(neg_data,0)
train_pos_num = int(pos_num*0.5+1)
train_neg_num = int(neg_num*0.5)
val_pos_num = int(pos_num*0.25+1)
val_neg_num = int(neg_num*0.25)
test_pos_num= int(pos_num*0.25)
test_neg_num = int(neg_num*0.25)
train_pos_num + val_pos_num + test_pos_num, train_neg_num + val_neg_num + test_neg_num

(6879, 1176868)

In [79]:
train_pos = pos_data.iloc[0:train_pos_num]
train_pos.shape

(3440, 67)

In [80]:
val_pos = pos_data.iloc[train_pos_num:train_pos_num+val_pos_num]
val_pos.shape

(1720, 67)

In [81]:
test_pos = pos_data.iloc[train_pos_num+val_pos_num:train_pos_num+val_pos_num+test_pos_num]
test_pos.shape

(1719, 67)

In [86]:
train_neg = neg_data.iloc[0:train_neg_num]
train_neg.shape

(588434, 67)

In [87]:
val_neg = neg_data.iloc[train_neg_num:train_neg_num+val_neg_num]
val_neg.shape

(294217, 67)

In [88]:
test_neg = neg_data.iloc[train_neg_num+val_neg_num:train_neg_num+val_neg_num+test_neg_num]
test_neg.shape

(294217, 67)

In [120]:
train_pos_id = train_pos[['Id']]
train_neg_id = train_neg[['Id']]
val_pos_id = val_pos[['Id']]
val_neg_id = val_neg[['Id']]
test_pos_id = test_pos[['Id']]
test_neg_id = test_neg[['Id']]

In [122]:
pos_val = train_numeric_reduced.loc[train_numeric_reduced['Id'].isin(val_pos_id['Id'])]
np.size(pos_val,0)

pos_val_data = pos_val.loc[pos_val['Response'] == 1] 
pos_val_data.shape

(1720, 67)

In [123]:
neg_val = train_numeric_reduced.loc[train_numeric_reduced['Id'].isin(val_neg_id['Id'])]
np.size(neg_val,0)

neg_val_data = neg_val.loc[neg_val['Response'] == 0] 
np.size(neg_val_data, 0)

294217

In [124]:
pos_train = train_numeric_reduced.loc[train_numeric_reduced['Id'].isin(train_pos_id['Id'])]
pos_train_data = pos_train.loc[pos_train['Response'] == 1] 
np.size(pos_train_data,0)

3440

In [125]:
neg_train = train_numeric_reduced.loc[train_numeric_reduced['Id'].isin(train_neg_id['Id'])]
neg_train_data = neg_train.loc[neg_train['Response'] == 0] 
np.size(neg_train_data,0)

588434

In [126]:
pos_test = train_numeric_reduced.loc[train_numeric_reduced['Id'].isin(test_pos_id['Id'])]
pos_test_data = pos_test.loc[pos_test['Response'] == 1] 
np.size(pos_test_data,0)

1719

In [127]:
neg_test = train_numeric_reduced.loc[train_numeric_reduced['Id'].isin(test_neg_id['Id'])]
neg_test_data = neg_test.loc[neg_test['Response'] == 0] 
np.size(neg_test_data,0)

294217

In [113]:
# train_pos_id = np.append('Id',train_pos_id)
# train_neg_id = np.append('Id',train_neg_id)
# val_pos_id = np.append('Id', val_pos_id)
# val_neg_id = np.append('Id', val_neg_id)
# test_pos_id = np.append('Id', test_pos_id)
# test_neg_id = np.append('Id',test_neg_id)

In [128]:
train_pos_id.to_csv('data/NewDataSet/train_pos_id.csv', index=False)
train_neg_id.to_csv('data/NewDataSet/train_neg_id.csv', index=False)
val_pos_id.to_csv('data/NewDataSet/val_pos_id.csv', index=False)
val_neg_id.to_csv('data/NewDataSet/val_neg_id.csv', index=False)
test_neg_id.to_csv('data/NewDataSet/test_neg_id.csv', index=False)
test_neg_id.to_csv('data/NewDataSet/test_neg_id.csv', index=False)

In [133]:
train_id = train_pos_id.append(train_neg_id)
train_id.to_csv('data/NewDataSet/train_id.csv', index=False)

train_pos_id.shape, train_neg_id.shape, train_id.shape

((3440, 1), (588434, 1), (591874, 1))

In [134]:
val_id = val_pos_id.append(val_neg_id)
val_id.to_csv('data/NewDataSet/val_id.csv', index=False)
test_id = test_pos_id.append(test_neg_id)
test_id.to_csv('data/NewDataSet/test_id.csv', index=False)

In [136]:
ftrain_pos_id = pd.read_csv('data/NewDataSet/train_pos_id.csv')
ftrain_neg_id = pd.read_csv('data/NewDataSet/train_neg_id.csv')
fval_pos_id = pd.read_csv('data/NewDataSet/val_pos_id.csv')
fval_neg_id = pd.read_csv('data/NewDataSet/val_neg_id.csv')
ftest_pos_id = pd.read_csv('data/NewDataSet/test_pos_id.csv')
ftest_neg_id = pd.read_csv('data/NewDataSet/test_neg_id.csv')
ftrain_id = pd.read_csv('data/NewDataSet/train_id.csv')
fval_id = pd.read_csv('data/NewDataSet/val_id.csv')
ftest_id = pd.read_csv('data/NewDataSet/test_id.csv')

In [140]:
# train_pos = pos_data.iloc[0:train_pos_num]
# np.size(train_pos,0)
ftrain_pos = pos_data.loc[pos_data['Id'].isin(ftrain_id['Id'])]
ftrain_neg = neg_data.loc[neg_data['Id'].isin(ftrain_id['Id'])]
np.size(ftrain_pos, 0), np.size(ftrain_neg, 0)

(3440, 588434)

In [141]:
# train_neg_num = int(neg_num*0.5)
# train_neg = neg_data.iloc[0:train_neg_num]
# np.size(train_neg, 0)
fval_pos = pos_data.loc[pos_data['Id'].isin(fval_id['Id'])]
fval_neg = neg_data.loc[neg_data['Id'].isin(fval_id['Id'])]
np.size(fval_pos, 0), np.size(fval_neg,0)

(1720, 294217)

In [142]:
ftest_pos = pos_data.loc[pos_data['Id'].isin(fval_id['Id'])]
ftest_neg = neg_data.loc[neg_data['Id'].isin(fval_id['Id'])]
np.size(ftest_pos, 0), np.size(ftest_neg,0)

(1720, 294217)

### Down Sampling

In [144]:
test = test_pos.append(test_neg)
test.shape

(295936, 67)

In [145]:
y_true = test['Response'].values

In [146]:
ratio = [1, 3, 10, 171] #, 20, 30, 40, 50, 100, 150, 171]
listp=[x for x in pos_data.columns if x not in ["Id","Response"]]

In [148]:
res_columns = ['pn_ratio', 'train_num', 'val_num', 'test_num', 'fpr0','fpr1', 'tpr0','tpr1', 'auc', 'mcc']
res = pd.DataFrame(columns = res_columns)

In [147]:
param = {'max_depth':10, 'eta':.1, 'silent':1}
param['colsample_bytree'] = 0.8
param['subsample'] = 0.8
param['min_child_weight'] = 3
param['base_score'] = 0.005
param['objective'] = "binary:logistic"
param['n_estimators']=200
param['learning_rate'] = 0.05
#param['eval_metric'] = "auc"

In [149]:
def mcc(tp, tn, fp, fn):
    sup = (tp * tn - fp * fn)*1.0
    inf = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    if inf == 0:
        return 0
    else:
        return sup / np.sqrt(inf)

def eval_mcc(y_true, y_prob, show=False):
    idx = np.argsort(y_prob)
    y_true_sort = y_true[idx]
    n = y_true.shape[0]
    nump = 1.0 * np.sum(y_true)  # number of positive
    numn = n - nump  # number of negative
    tp = nump
    tn = 0.0
    fp = numn
    fn = 0.0
    best_mcc = 0.0
    best_id = -1
    mccs = np.zeros(n)
    for i in range(n):
        if y_true_sort[i] == 1:
            tp -= 1.0
            fn += 1.0
        else:
            fp -= 1.0
            tn += 1.0
        new_mcc = mcc(tp, tn, fp, fn)
        mccs[i] = new_mcc
        if new_mcc >= best_mcc:
            best_mcc = new_mcc
            best_id = i
    
    #print(mccs)
    if show:
        best_proba = y_prob[idx[best_id]]
        y_pred = (y_prob > best_proba).astype(int)
        return best_proba, best_mcc, y_pred
    else:
        return best_mcc


def mcc_eval(y_prob, dtrain):
    y_true = dtrain.get_label()
    best_mcc = eval_mcc(y_true, y_prob)
    return 'MCC', best_mcc

(294217, 67)

In [157]:
for i in ratio:
    print('** Pos-Neg Ratio = 1:{0}'.format(i))
    train_neg_num = train_pos_num * i
    val_neg_num = val_pos_num * i
    train = train_pos.append(train_neg.iloc[0:train_neg_num])
    val = val_pos.append(val_neg.iloc[0:val_neg_num])
    Xtrain=train[listp]
    Ytrain=train['Response']
    Xval = val[listp]
    Yval = val['Response']
    dvisibletrain = xgb.DMatrix(Xtrain,Ytrain,silent=True)
    dvisibleval = xgb.DMatrix(Xval,Yval, silent=True)
    watchlist = [(dvisibletrain, 'train'), (dvisibleval, 'val')]
    ## how many trees 
    num_round=100
    clf=xgb.train(param, dvisibletrain, num_round, 
                  evals=watchlist,
                  early_stopping_rounds=50,
                  feval = mcc_eval,
                  maximize=True)
    dtest = xgb.DMatrix(test[listp],silent=True)
    prediction = clf.predict(dtest)
    y_pred = (prediction > .08).astype(int)
    fpr, tpr, _ = metrics.roc_curve(y_true, y_pred)
    auc = metrics.auc(fpr, tpr)
    mcc = eval_mcc(y_true, y_pred)
    res.loc[len(res)]= [i, np.size(train,0), np.size(val,0), np.size(test,0), fpr[0], fpr[1], tpr[0], tpr[1], auc, mcc]

** Pos-Neg Ratio = 1:1


TypeError: 'numpy.float64' object is not callable

In [85]:
res.to_csv('results_f65.csv', index=False)

In [155]:
res

,pn_ratio,train_num,val_num,test_num,fpr0,fpr1,tpr0,tpr1,auc
0,1.0,6880.0,3440.0,295936.0,0.0,0.999541,0.0,1.000000,0.500229
1,3.0,13760.0,6880.0,295936.0,0.0,0.914852,0.0,0.954043,0.519596
2,10.0,37840.0,18920.0,295936.0,0.0,0.376426,0.0,0.502618,0.563096
3,20.0,72240.0,36120.0,295936.0,0.0,0.096996,0.0,0.180337,0.541670
4,30.0,106640.0,53320.0,295936.0,0.0,0.032874,0.0,0.086097,0.526611
5,40.0,141040.0,70520.0,295936.0,0.0,0.014652,0.0,0.053519,0.519434
6,50.0,175440.0,87720.0,295936.0,0.0,0.008494,0.0,0.040721,0.516114
7,100.0,347440.0,173720.0,295936.0,0.0,0.001125,0.0,0.021524,0.510200
8,150.0,519440.0,259720.0,295936.0,0.0,0.000333,0.0,0.017452,0.508559
9,1.0,6880.0,3440.0,295936.0,0.0,1.000000,0.0,1.000000,0.500000


In [87]:
prec, rec, thres = metrics.precision_recall_curve(y_true, y_pred)
prec

array([ 0.00580869,  0.234375  ,  1.        ])

In [88]:
rec

array([ 1.        ,  0.01745201,  0.        ])

In [89]:
thres

array([0, 1])

In [91]:
#train_all = train_pos.append(train_neg)

NameError: name 'train_neg' is not defined

In [171]:
#id_feature=pd.read_csv('data/idfeature.csv')
#id_feature.shape

### Merge other tables

In [172]:
#train = pd.merge(train_numeric_reduced, id_feature, on='Id', how='left')
#train = train_numeric_reduced

## Train graident boosting tree

## Generate train table for xgb

## Training xgb

## Score on test data

In [177]:
#test=pd.read_csv('data/original/test_numeric.csv')
dtest = xgb.DMatrix(test[listp],silent=True)
prediction = clf.predict(dtest)
y_pred = (prediction > .08).astype(int)

# resultSubmission = pd.DataFrame({'Id':test['Id'],'Response':y_pred})
# resultSubmission.to_csv('submitResponse.csv',index=False)
# resultSubmission.shape
# resultSubmission.head()